# Timer

In [7]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 29, 16, 10

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            break
    sleep(60)

# Parameter optimization

In [8]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['PriceChange']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 50
load = False

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'PriceChange': {"low_price_quantile": [1 * (i + 1) for i in range(20)]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Number of combinations is 20


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [03:04<00:00,  9.24s/it]


# Check global statistics

In [9]:
from glob import glob

ttype = 'sell'
pattern = ['PriceChange']
work_timeframe = '15m'
higher_timeframe = '4h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['PriceChange_low_price_quantile']).agg({'pct_right_forecast_avg': 'mean',
                                                                   'pct_price_diff_avg': 'mean',
                                                                   'forecast_rank': 'mean', 
                                                                   'price_rank': 'mean', 
                                                                   'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                                        ascending=False)
total_stat.head(20)

,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank,forecasts_num
PriceChange_low_price_quantile,,,,,
19,72.948917,-0.985667,124.207958,-203.629333,3871
20,72.854708,-0.981625,121.781542,-205.891792,3892
18,72.567000,-0.982792,105.875583,-201.888417,3851
12,73.488792,-1.254583,101.896083,-194.220375,3731
11,73.602042,-1.405000,101.734583,-194.816625,3709
2,83.106875,-3.800042,98.381375,-180.667750,3564
17,72.704083,-1.024250,95.535167,-200.036333,3827
1,89.329000,-6.621958,89.547833,-180.693542,3553
16,72.668625,-1.048167,88.317458,-197.423625,3802


# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [20]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)